# Measuring Word Similarity with BERT (English Language Public Domain Poems)

By [The BERT for Humanists](https://melaniewalsh.github.io/BERT-for-Humanists/) Team

How can we measure the similarity of words in a collection of texts? For example, how similar are the words "nature" and "science" in a collection of 16th-20th century English language poems? Do 20th-century poets use the word "science" differently than 16th-century poets? Can we map all the different uses and meanings of the word "nature"?

The short answer is: yes! We'll use BERT contextual embeddings to explore the relationships between individual occurrences of words in a corpus. This differs from the static embeddings we've explored previously. Where static embeddings ecode all possible menaings of a word into a single vectore representation, contextual embeddings encode each use of a word differently, depending on the words around it (that is, depending on its context).

## In This Notebook
We will analyze a collection of poems scraped from [Public-Domain-Poetry.com](http://public-domain-poetry.com/) with the [DistilBert model](https://huggingface.co/transformers/model_doc/distilbert.html) and the HuggingFace Python library. DistilBert is a smaller — yet still powerful! — version of BERT. By using the rich representations of words that BERT produces, we will then explore the multivalent meanings of particular words in context and over time.

We hope this notebook will help illustrate how BERT works, how well it works, and how you might use BERT to explore the similarity of words in a collection of texts. It is surprising, for example, that BERT works as well as it does, without any fine-tuning, on poems that were published hundreds of years before the text data it was trained on (Wikiepdia pages and self-published novels).

But we also hope that these results will expose some of the limitations and challenges of BERT. We have to disregard poetic line breaks, for example, and we see that BERT has trouble with antiquated words like "thine," which don't show up in its contemporary vocabulary.

In [1]:
import pandas as pd
import altair as alt

url = "https://raw.githubusercontent.com/melaniewalsh/BERT-4-Humanists/main/data/bert-word-nature.csv"
df = pd.read_csv(url, encoding='utf-8')

search_keywords = ['nature', 'science', 'religion', 'art']
color_by = 'word'

alt.Chart(df, title=f"Word Similarity: {', '.join(search_keywords).title()}").mark_circle(size=200).encode(
    alt.X('x',
        scale=alt.Scale(zero=False)
    ), y="y",
    color= color_by,
    href="link",
    tooltip=['title', 'word', 'poem_title', 'author', 'period']
    ).interactive().properties(
    width=500,
    height=500
)

/Users/mw966/miniforge3/envs/3350/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/mw966/miniforge3/envs/3350/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/mw966/miniforge3/envs/3350/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/

alt.Chart(...)

The plot above displays a preview of our later results. This is what we're working toward!

You can hover over each point to see the instance of each word in context. If you press `Shift` and click on a point, you will be taken to the original poem on Public-Domain-Poetry.com. Try it out!

## **Import necessary Python libraries and modules**

Ok enough introduction! Let's get started.

We import the DistilBertModel and DistilBertTokenizerFast from the Hugging Face `transformers` library. We also import a handful of other Python libraries and modules.

In [ ]:
# new
from   transformers import DistilBertTokenizerFast, DistilBertModel
import altair as alt

# others
import numpy as np
import os
import pandas as pd
from   sklearn.decomposition import PCA

pd.options.display.max_colwidth = 200

## **Load text dataset**

Our dataset contains around 30,000 poems scraped from  http://public-domain-poetry.com/. This website hosts a curated collection of poems that have fallen out of copyright, which makes them easier to share.
You can find the data in the AI for Humanists [GitHub repository](https://github.com/melaniewalsh/BERT-4-Humanists/blob/main/data/public-domain-poetry.csv).

We don't have granular date information about when each poem was published, but we do know the birth dates of most of our authors, which we've used to loosely categorize the poems by time period. The poems in our data range from the Middle Ages to the twentieth century, but most come from the nineteenth century. The data features both well-known authors — William Wordsworth, Emily  Dickinson, Paul Laurence Dunbar, Walt Whitman, Shakespeare — as well as less well-known authors.

In [ ]:
# read poems from b4h github repo
url = "https://raw.githubusercontent.com/melaniewalsh/BERT-4-Humanists/main/data/public-domain-poetry.csv"
poetry_df = pd.read_csv(url, encoding='utf-8')
poetry_df.sample(5)

In [ ]:
# count of poems
len(poetry_df)

In [ ]:
# top authors
poetry_df['author'].value_counts()[:20]

In [ ]:
# vis counts by period
poetry_df['period'].sort_values().hist(figsize=(15, 5));

## **Sample text dataset**

Though we wish we could analyze all the poems in this data, Colab tends to crash if we try to use more than 4-5,000 poems —  even with DistilBert, the smaller version of BERT. This is an important limitation to keep in mind. If you'd like to use more text data, you might consider upgrading to a paid version of Colab (with more memory or GPUs) or using a compute cluster.

To reduce the number of poems, we will take a random sample of 1,000 poems from four different time periods: the 20th Century, 19th Century, 18th Century, and the Early Modern period.

In [ ]:
# Filter the DataFrame for only a given time period, then randomly sample 1000 rows
nineteenth_sample = poetry_df[poetry_df['period'] == '19th Century'].sample(1000)
twentieth_sample = poetry_df[poetry_df['period'] == '20th Century'].sample(1000)
eighteenth_sample = poetry_df[poetry_df['period'] == '18th Century'].sample(1000)
sixteenth_sample = poetry_df[poetry_df['period'] == '16th-17th Centuries (Early Modern)'].sample(1000)

In [ ]:
# Merge these random samples into a new DataFrame
poetry_df = pd.concat([sixteenth_sample, eighteenth_sample, twentieth_sample, nineteenth_sample])

In [ ]:
poetry_df['period'].value_counts()

Finally, let's make a list of poems from our Pandas DataFrame.

In [ ]:
poetry_texts = poetry_df['text'].tolist()

Let's examine a poem in our dataset:

In [ ]:
len(poetry_texts)

In [ ]:
print(poetry_texts[2])

## **Encode/tokenize text data for BERT**

Next we need to transform our poems into a format that BERT (via Huggingface) will understand. This is called *tokenizing* the data.

We will tokenize the poems with the `tokenizer()` from HuggingFace's `DistilBertTokenizerFast`. Here's what the `tokenizer()` will do:

1. Truncate the texts if they're more than 512 tokens or pad them if they're fewer than 512 tokens. If a word is not in BERT's vocabulary, it will be broken up into smaller "word pieces," demarcated by a `##`.

2. Add in special tokens to help BERT:
    - [CLS] — Start token of every document
    - [SEP] — Separator between each sentence
    - [PAD] — Padding at the end of the document as many times as necessary, up to 512 tokens
    - &#35;&#35; — Start of a "word piece"

Here we will load `DistilBertTokenizerFast` from `transformers` library, which will help us transform and encode the texts so they can be used with BERT.

In [ ]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

The `tokenizer()` will break word tokens into word pieces, truncate to 512 tokens, and add padding and special BERT tokens.

In [ ]:
tokenized_poems = tokenizer(poetry_texts, truncation=True, padding=True, return_tensors="pt")

Let's examine the first tokenized poem. We can see that the special BERT tokens have been inserted where necessary.

In [ ]:
' '.join(tokenized_poems[2].tokens)

## **Load pre-trained BERT model**

Here we will load a pre-trained BERT model. To speed things up we will use a GPU, but using GPU involves a few extra steps.
The command `.to("cuda")` moves data from regular memory to the GPU's memory.




In [ ]:
#device = 'mps' # apple silicon gpu
device = 'cuda' # nvidia gpu
#device = 'cpu' # slooooow

In [ ]:
model = DistilBertModel.from_pretrained('distilbert-base-uncased').to(device)

In [ ]:
# check device in use
model.device

## **Get BERT word embeddings for each document in a collection**

For each poem in our list `poetry_texts`, we will tokenize the poem, and we will extract the vocabulary word ID for each word/token in the poem (to use for later reference). Then we will run the tokenized poem through the BERT model and extract the contextual vectors for each word/token in the poem.

We thus create two big lists for all the poems in our collection — `doc_word_ids` and `doc_word_vectors`.

In [ ]:
%%time
# List of vocabulary word IDs for all the words in each document (aka each poem)
doc_word_ids = []
# List of word vectors for all the words in each document (aka each poem)
doc_word_vectors = []

# slice each poem to ignore the first (0th) and last (-1) special BERT tokens
# generally do not want to do this, but here we only want the regular embeddings
start_of_words = 1
end_of_words = -1

# Below we will index the 0th or first document, which will be the only document, since we're analzying one poem at a time
first_document = 0

for i, poem in enumerate(poetry_texts):

    # Here we tokenize each poem with the DistilBERT Tokenizer
    inputs = tokenizer(poem, return_tensors="pt", truncation=True, padding=True)

    # Here we extract the vocabulary word ids for all the words in the poem (the first or 0th document, since we only have one document)
    # We ignore the first and last special BERT tokens
    # We also convert from a Pytorch tensor to a numpy array
    doc_word_ids.append(inputs.input_ids[first_document].numpy()[start_of_words:end_of_words])

    # Here we send the tokenized poems to the GPU
    # The model is already on the GPU, but this poem isn't, so we send it to the GPU
    inputs.to(device)
    # Here we run the tokenized poem through the DistilBERT model
    outputs = model(**inputs)

    # We take every element from the first or 0th document, from the 2nd to the 2nd-to-last position
    # Grabbing the last layer is one way of getting token vectors. There are different ways to get vectors with different pros and cons
    doc_word_vectors.append(outputs.last_hidden_state[first_document,start_of_words:end_of_words,:].detach().cpu().numpy())


Confirm that we have the same number of documents for both the tokens and the vectors:

In [ ]:
len(doc_word_ids), len(doc_word_vectors)

In [ ]:
doc_word_ids[2], doc_word_vectors[2]

## **Concatenate all word IDs/vectors for all documents**

Each element of these lists contains all the tokens/vectors for one document. But we want to concatenate them into two giant collections.

In [ ]:
all_word_ids = np.concatenate(doc_word_ids)
all_word_vectors = np.concatenate(doc_word_vectors, axis=0)

We want to make comparisons between vectors quickly. We'll use cosine similarity, or the dot product of `l2`-normed vectors. So we need to norm our embeddings.

In [ ]:
# normalize embeddings
row_norms = np.sqrt(np.sum(all_word_vectors ** 2, axis=1))
all_word_vectors /= row_norms[:,np.newaxis]

## **Find all word positions in a collection**

We can use the array `all_word_ids` to find all the places, or *positions*, in the collection where a word appears.

We can find a word's vocab ID in BERT with `tokenizer.vocab` and then check to see where/how many times this ID occurs in `all_word_ids`.

In [ ]:
def get_word_positions(words):
    """This function accepts a list of words, rather than a single word"""

    # Get word/vocabulary ID from BERT for each word
    word_ids = [tokenizer.vocab[word] for word in words]

    # Find all the positions where the words occur in the collection
    word_positions = np.where(np.isin(all_word_ids, word_ids))[0]

    return word_positions

Here we'll check to see all the places where the word "bank" appears in the collection.

In [ ]:
get_word_positions(["bank"])

In [ ]:
word_positions = get_word_positions(["bank"])

## **Find word from word position**

Nice! Now we know all the positions where the word "bank" appears in the collection. But it would be more helpful to know the actual words that appear in context around it. To find these context words, we have to convert position IDs back into words.

In [ ]:
# Here we create an array so that we can go backwards from numeric token IDs to words
word_lookup = np.empty(tokenizer.vocab_size, dtype="O")

for word, index in tokenizer.vocab.items():
    word_lookup[index] = word

Now we can use `word_lookup` to find a word based on its position in the collection.

In [ ]:
word_positions = get_word_positions(["bank"])

for word_position in word_positions:
    print(f'{word_position:>10} {word_lookup[all_word_ids[word_position]]}')

We can also look for the 3 words that come before "bank" and the 3 words that come after it.

In [ ]:
word_positions = get_word_positions(["bank"])

for word_position in word_positions:

    # Slice 3 words before "bank"
    start_pos = word_position - 3
    # Slice 3 words after "bank"
    end_pos = word_position + 4

    context_words = word_lookup[all_word_ids[start_pos:end_pos]]
    # Join the words together
    context_words = ' '.join(context_words)
    print(word_position, context_words)

Let's make some functions that will help us get the context words around a certain word position for whatever size window (certain number of words before and after) that we want.

The first function `get_context()` will simply return the tokens without cleaning them, and the second function `get_context_clean()` will return the tokens in a more readable fashion.

In [ ]:
def get_context(word_id, window_size=10):

    """Simply get the tokens that occur before and after word position"""

    start_pos = max(0, word_id - window_size) # The token where we will start the context view
    end_pos = min(word_id + window_size + 1, len(all_word_ids)) # The token where we will end the context view

    # Make a list called tokens and use word_lookup to get the words for given token IDs from starting position up to the keyword
    tokens = [word_lookup[word] for word in all_word_ids[start_pos:end_pos] ]

    context_words = " ".join(tokens)

    return context_words

In [ ]:
import re

def get_context_clean(word_id, window_size=10):

    """Get the tokens that occur before and after word position AND make them more readable"""

    keyword = word_lookup[all_word_ids[word_id]]
    start_pos = max(0, word_id - window_size) # The token where we will start the context view
    end_pos = min(word_id + window_size + 1, len(all_word_ids)) # The token where we will end the context view

    # Make a list called tokens and use word_lookup to get the words for given token IDs from starting position up to the keyword
    tokens = [word_lookup[word] for word in all_word_ids[start_pos:end_pos] ]

    # Make wordpieces slightly more readable
    # This is probably not the most efficient way to clean and correct for weird spacing
    context_words = " ".join(tokens)
    context_words = re.sub(r'\s+([##])', r'\1', context_words)
    context_words = re.sub(r'##', r'', context_words)
    context_words = re.sub('\s+\'s', '\'s', context_words)
    context_words = re.sub('\s+\'d', '\'d', context_words)
    context_words = re.sub('\s\'er', '\'er', context_words)
    context_words = re.sub(r'\s+([-,:?.!;])', r'\1', context_words)
    context_words = re.sub(r'([-\'"])\s+', r'\1', context_words)
    context_words = re.sub('\s+\'s', '\'s', context_words)
    context_words = re.sub('\s+\'d', '\'d', context_words)

    # Bold the keyword by putting asterisks around it
    if keyword in context_words:
        context_words = re.sub(f"\\b{keyword}\\b", f"**{keyword}**", context_words)
        context_words = re.sub(f"\\b({keyword}[esdtrlying]+)\\b", fr"**\1**", context_words)

    return context_words

To visualize the search keyword even more easily, we're going to import a couple of Python modules that will allow us to output text with bolded words and other styling. Here we will make a function `print_md()` that will allow us to print with Markdown styling.

In [ ]:
from IPython.display import Markdown, display

def print_md(string):
    display(Markdown(string))

In [ ]:
word_positions = get_word_positions(["bank"])
for word_position in word_positions:
    print_md(f"{word_position}:  {get_context_clean(word_position)}")

Here we make a list of all the context views for our keyword.

In [ ]:
word_positions = get_word_positions(["bank"])

keyword_contexts = []
keyword_contexts_tokens = []

for position in word_positions:
    keyword_contexts.append(get_context_clean(position))
    keyword_contexts_tokens.append(get_context(position))

## **Get word vectors and reduce them with PCA**

Finally, we don't just want to *read* all the instances of "bank" in the collection, we want to *measure* the similarity of all the instances of "bank."

To measure similarity between all the instances of "bank," we will take the vectors for each instance and then use PCA to reduce each 768-dimensionsal vector to the 2 dimensions that capture the most variation.

In [ ]:
word_positions = get_word_positions(["bank"])
pca = PCA(n_components=2)
pca.fit(all_word_vectors[word_positions,:].T)

Then, for convenience, we will put these PCA results into a Pandas DataFrame, which will use to generate an interactive plot.

In [ ]:
df = pd.DataFrame({"x": pca.components_[0,:], "y": pca.components_[1,:],
                   "context": keyword_contexts, "tokens": keyword_contexts_tokens})
df.head()

## **Match context with original text and metadata**

It's helpful (and fun!) to know where each instance of a word actually comes from — which poem, which poet, which time period, which Public-Domain-Poetry.com web page. The easiest method we've found for matching a bit of context with its original poem and metdata is to 1) add a tokenized version of each poem to our original Pandas Dataframe 2) check to see if the context shows up in a poem 3) and if so, grab the original poem and metadata.

In [ ]:
# Tokenize all the poems
tokenized_poems = tokenizer(poetry_texts, truncation=True, padding=True, return_tensors="pt")

# Get a list of all the tokens for each poem
all_tokenized_poems = []
for i in range(len(tokenized_poems['input_ids'])):
    all_tokenized_poems.append(' '.join(tokenized_poems[i].tokens))

# Add them to the original DataFrame
poetry_df['tokens'] = all_tokenized_poems

In [ ]:
poetry_df.head(2)

In [ ]:
def find_original_poem(rows):

    """This function checks to see whether the context tokens show up in the original poem,
    and if so, returns metadata about the title, author, period, and URL for that poem"""

    text = rows['tokens'].replace('**', '')
    text = text[55:70]

    if poetry_df['tokens'].str.contains(text, regex=False).any() == True :
        row = poetry_df[poetry_df['tokens'].str.contains(text, regex=False)].values[0]
        title, author, period, link = row[0], row[1], row[7], row[6]
        return author, title, period, link
    else:
        return None, None, None, None

In [ ]:
df[['title', 'author', 'period', 'link']] = df.apply(find_original_poem, axis='columns', result_type='expand')

In [ ]:
df.head()

## **Plot word embeddings**

Lastly, we will plot the words vectors from this DataFrame with the Python data viz library [Altair](https://altair-viz.github.io/gallery/scatter_tooltips.html).

In [ ]:
alt.Chart(df,title="Word Similarity: Bank").mark_circle(size=200).encode(
    alt.X('x',
        scale=alt.Scale(zero=False)
    ), y="y",
    # If you click a point, take you to the URL link
    href="link",
    # The categories that show up in the hover tooltip
    tooltip=['title', 'context', 'author', 'period']
    ).interactive().properties(
    width=500,
    height=500
)

## **Plot word embeddings from keywords (all at once!)**

We can put the code from the previous few sections into a single cell and plot the BERT word embeddings for any list of words. Let's look at the words "nature," "religion," "science," and "art."

In [ ]:
# List of keywords that you want to compare
keywords = ['nature', 'religion', 'science', 'art']

# How to color the points in the plot. The other option is "period" for time period
color_by = 'word'

# Get all word positions
word_positions = get_word_positions(keywords)

# Get all contexts around the words
keyword_contexts = []
keyword_contexts_tokens = []
words = []

for position in word_positions:
    words.append(word_lookup[all_word_ids[position]])
    keyword_contexts.append(get_context_clean(position))
    keyword_contexts_tokens.append(get_context(position))

# Reduce word vectors with PCA
pca = PCA(n_components=2)
pca.fit(all_word_vectors[word_positions,:].T)

# Make a DataFrame with PCA results
df = pd.DataFrame({"x": pca.components_[0,:], "y": pca.components_[1,:],
                   "context": keyword_contexts, "tokens": keyword_contexts_tokens, "word": words})
# Match original text and metadata
df[['title', 'author', 'period', 'link']] = df.apply(find_original_poem, axis='columns', result_type='expand')

# Rename columns so that the context shows up as the "title" in the tooltip (bigger and bolded)
df = df.rename(columns={'title': 'poem_title', 'context': 'title'})

# Make the plot
alt.Chart(df, title=f"Word Similarity: {', '.join(keywords).title()}").mark_circle(size=200).encode(
    alt.X('x',
        scale=alt.Scale(zero=False)
    ), y="y",
    color= color_by,
    href="link",
    tooltip=['title', 'word', 'poem_title', 'author', 'period']
    ).interactive().properties(
    width=500,
    height=500
)

Let's examine the words "nature," "religion," "science," and "art" again but this time color the points by their time period.

In [ ]:
# List of keywords that you want to compare
keywords = ['nature', 'religion', 'science', 'art']

# How to color the points in the plot
color_by = 'period'

# Get all word positions
word_positions = get_word_positions(keywords)

# Get all contexts around the words
keyword_contexts = []
keyword_contexts_tokens = []
words = []

for position in word_positions:
    words.append(word_lookup[all_word_ids[position]])
    keyword_contexts.append(get_context_clean(position))
    keyword_contexts_tokens.append(get_context(position))

# Reduce word vectors with PCA
pca = PCA(n_components=2)
pca.fit(all_word_vectors[word_positions,:].T)

# Make a DataFrame with PCA results
df = pd.DataFrame({"x": pca.components_[0,:], "y": pca.components_[1,:],
                   "context": keyword_contexts, "tokens": keyword_contexts_tokens, "word": words})
# Match original text and metadata
df[['title', 'author', 'period', 'link']] = df.apply(find_original_poem, axis='columns', result_type='expand')

# Rename columns so that the context shows up as the "title" in the tooltip (bigger and bolded)
df = df.rename(columns={'title': 'poem_title', 'context': 'title'})

# Make the plot
alt.Chart(df, title=f"Word Similarity: {', '.join(keywords).title()}").mark_circle(size=200).encode(
    alt.X('x',
        scale=alt.Scale(zero=False)
    ), y="y",
    color= color_by,
    href="link",
    tooltip=['title', 'word', 'poem_title', 'author', 'period']
    ).interactive().properties(
    width=500,
    height=500
)

Let's compare the words "mean," "thin," "average", and "cruel."

In [ ]:
# List of keywords that you want to compare
keywords = ['mean', 'thin', 'average', 'cruel']

# How to color the points in the plot. The other option is "period" for time period
color_by = 'word'

# Get all word positions
word_positions = get_word_positions(keywords)

# Get all contexts around the words
keyword_contexts = []
keyword_contexts_tokens = []
words = []

for position in word_positions:
    words.append(word_lookup[all_word_ids[position]])
    keyword_contexts.append(get_context_clean(position))
    keyword_contexts_tokens.append(get_context(position))

# Reduce word vectors with PCA
pca = PCA(n_components=2)
pca.fit(all_word_vectors[word_positions,:].T)

# Make a DataFrame with PCA results
df = pd.DataFrame({"x": pca.components_[0,:], "y": pca.components_[1,:],
                   "context": keyword_contexts, "tokens": keyword_contexts_tokens, "word": words})
# Match original text and metadata
df[['title', 'author', 'period', 'link']] = df.apply(find_original_poem, axis='columns', result_type='expand')

# Rename columns so that the context shows up as the "title" in the tooltip (bigger and bolded)
df = df.rename(columns={'title': 'poem_title', 'context': 'title'})

# Make the plot
alt.Chart(df, title=f"Word Similarity: {', '.join(keywords).title()}").mark_circle(size=200).encode(
    alt.X('x',
        scale=alt.Scale(zero=False)
    ), y="y",
    color= color_by,
    href="link",
    tooltip=['title', 'word', 'poem_title', 'author', 'period']
    ).interactive().properties(
    width=500,
    height=500
)

See the difference between our static embeddings and these contextual ones

In [ ]:
# List of keywords that you want to compare
keywords = ['bank', 'river', 'money', 'coffee', 'kitten']

# How to color the points in the plot. The other option is "period" for time period
color_by = 'word'

# Get all word positions
word_positions = get_word_positions(keywords)

# Get all contexts around the words
keyword_contexts = []
keyword_contexts_tokens = []
words = []

for position in word_positions:
    words.append(word_lookup[all_word_ids[position]])
    keyword_contexts.append(get_context_clean(position))
    keyword_contexts_tokens.append(get_context(position))

# Reduce word vectors with PCA
pca = PCA(n_components=2)
pca.fit(all_word_vectors[word_positions,:].T)

# Make a DataFrame with PCA results
df = pd.DataFrame({"x": pca.components_[0,:], "y": pca.components_[1,:],
                   "context": keyword_contexts, "tokens": keyword_contexts_tokens, "word": words})
# Match original text and metadata
df[['title', 'author', 'period', 'link']] = df.apply(find_original_poem, axis='columns', result_type='expand')

# Rename columns so that the context shows up as the "title" in the tooltip (bigger and bolded)
df = df.rename(columns={'title': 'poem_title', 'context': 'title'})

# Make the plot
alt.Chart(df, title=f"Word Similarity: {', '.join(keywords).title()}").mark_circle(size=200).encode(
    alt.X('x',
        scale=alt.Scale(zero=False)
    ), y="y",
    color= color_by,
    href="link",
    tooltip=['title', 'word', 'poem_title', 'author', 'period']
    ).interactive().properties(
    width=500,
    height=500
)

## **Find word similarity from a specific word position**

We can also search *all* of the vectors for words similar to a query word.

In [ ]:
def get_nearest(query_vector, n=20):
    cosines = all_word_vectors.dot(query_vector)
    ordering = np.flip(np.argsort(cosines))
    return ordering[:n]

To do so, we need to find the specific word position of our desired search keyword.

In [ ]:
word_positions = get_word_positions(['bank'])

for word_position in word_positions:
    print_md(f"{word_position}: {get_context_clean(word_position)}")

In [ ]:
keyword_position = word_positions[-1] # pick last of the sampled instances above

In [ ]:
contexts = [get_context_clean(token_id) for token_id in get_nearest(all_word_vectors[keyword_position,:])]

for context in contexts:
    print_md(context)